In [1]:
!python --version
!pip --version

Python 3.6.5 :: Anaconda, Inc.


pip 10.0.1 from C:\Users\Brian\Anaconda3\lib\site-packages\pip (python 3.6)



In [2]:
!pip install beautifulsoup4
!pip install pandas
!pip install datetime

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import re

In [10]:
def monthToNum(shortMonth):
    return{
            'January' : 1,
            'February' : 2,
            'March' : 3,
            'April' : 4,
            'May' : 5,
            'June' : 6,
            'July' : 7,
            'August' : 8,
            'September' : 9, 
            'October' : 10,
            'November' : 11,
            'December' : 12
    }[shortMonth]
def getOdds(data):
    odds = data.str.split(',')
    odds = odds[0][0]
    odds = float(odds.split(': Win ')[1])
    return odds
def getLine(data):
    line = data.str.split(',')[0][1]
    line = re.sub(' @ 1.92','', re.sub(' Line ', '', line))
    return line
def getSoup(url):
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [11]:
def infoTable(some_soup):
    info_table = some_soup.find('table', border="0", cellpadding="0", cellspacing="0", width="375") #find the first table with this info
    info_table = pd.read_html(str(info_table))
    info_table = info_table[0]
    return info_table
def dateTime(the_info_table):
    date_time = the_info_table.iloc[2]
    date_time = date_time.str.split(',')
    date = date_time[0][1].split(' ')
    day = re.sub('[^0-9]','',date[1])
    year = date[3]
    month = monthToNum(date[2])
    time = date_time[0][2].split(' ')
    date = datetime.datetime(int(year), month, int(day))
    return date
def getGameInfo(the_info_table):
    game_info = the_info_table.iloc[1]
    game_info = game_info.str.split(',')
    return game_info
def getStatTable(some_soup):
    info_table = some_soup.find('table', border="0", cellpadding="0", cellspacing="0", width="375") #find the first table with this info
    stat_table = info_table.find_next('table', border="0", cellpadding="0", cellspacing="0", width="375") #find the next table same info
    stat_table = pd.read_html(str(stat_table))[0]
    return stat_table
def getStats(the_stat_table, team, stat_type):
    if team == 'home':
        col_num = 0
    if team == 'away':
        col_num = 2
    if stat_type == 'basic':
        last_row = 27
    if stat_type == 'advanced':
        last_row = 22
    stats = pd.Series(data =  the_stat_table.iloc[2:last_row,col_num]).str.replace('%','').astype('float')
    return stats
def replacePcnt(data):
    data = data.str.replace('% ','').str.replace(' %','').str.replace('%','').str.replace(' ', '_')
    return data

In [12]:
afl_url = 'https://www.footywire.com/afl/footy/ft_match_statistics?mid=9516'
soup = getSoup(afl_url)

In [28]:
info_table = infoTable(soup)
date = dateTime(info_table)
year = date.year

game_info = getGameInfo(info_table)
season_round = int(re.sub('[^0-9]','',game_info[0][0]))
location = game_info[0][1]
attendance = int(re.sub(' Attendance: ', '', game_info[0][2]))

home_odds = getOdds(info_table.iloc[3])
away_odds = getOdds(info_table.iloc[4])
home_line = getLine(info_table.iloc[3])
away_line = getLine(info_table.iloc[4])

In [23]:
bsc_stat_tbl = getStatTable(soup)
bsc_home_stats = getStats(bsc_stat_tbl, 'home', 'basic')
bsc_away_stats = getStats(bsc_stat_tbl, 'away', 'basic')
bsc_stat_lbl = replacePcnt(pd.Series(data =  bsc_stat_tbl.iloc[2:27,1]))
home_team = bsc_stat_tbl.iloc[1,1]
away_team = bsc_stat_tbl.iloc[1,3]

In [24]:
afl_adv_url = afl_url + '&advv=Y'
advanced_soup = getSoup(afl_adv_url)

In [25]:
adv_stat_tbl = getStatTable(advanced_soup)
adv_home_stats = getStats(adv_stat_tbl, 'home', 'advanced')
adv_away_stats = getStats(adv_stat_tbl, 'away', 'advanced')
adv_stat_lbl = replacePcnt(pd.Series(data =  adv_stat_tbl.iloc[2:22,1]))

In [26]:
header_cols = pd.Series(['season_round', 'year', 'date', 'location', 'attendance', 
           'home_odds', 'away_odds', 'home_line', 'away_line'])
header_cols = pd.concat([header_cols,
          bsc_stat_lbl + 'Home', adv_stat_lbl + 'Home',
           bsc_stat_lbl + 'Away', adv_stat_lbl + 'Away'], 
         axis = 0).reset_index(drop=True)
afl_DF = pd.DataFrame(columns = header_cols)

In [29]:
game_meta = pd.Series([season_round, year, date, location, attendance, 
           home_odds, away_odds, home_line, away_line])
game_stats = pd.concat([bsc_home_stats, adv_home_stats, bsc_away_stats, adv_away_stats])
game_data = pd.concat([game_meta, game_stats])
game_data.index = header_cols

In [30]:
afl_DF = afl_DF.append(game_data, ignore_index=True)
afl_DF

,season_round,year,date,location,attendance,home_odds,away_odds,home_line,away_line,KicksHome,...,BouncesAway,Goal_AssistsAway,Goals_AssistedAway,Centre_ClearancesAway,Stoppage_ClearancesAway,Score_InvolvementsAway,Metres_GainedAway,TurnoversAway,InterceptsAway,Tackles_Inside_50Away
0,1,2018,2018-03-24,Marvel Stadium,23731,1.33,3.45,-22.5,+22.5,242.0,...,3.0,8.0,66.7,17.0,27.0,89.0,5054.0,68.0,61.0,10.0
